Gábor Horváth, Illés Horváth, Salah Al-Deen Almousa, Miklós Telek
# Inverse Laplace Transform with Concentrated Matrix-Exponential Functions

## Implementation of the inverse Laplace transform procedure

Note: the "iltcme.json" (containing the coefficients of the CME functions) must be in the working folder

In [1]:
import math
import cmath
import json
import numpy as np

def ilt(fun, T, maxFnEvals, method="cme"):
    if method=="cme":
        if "cmeParams" not in globals():
            with open('iltcme.json') as f:
                globals()["cmeParams"] = json.load(f)
        # find the most steep CME satisfying maxFnEvals
        params = cmeParams[0]
        for p in cmeParams:
            if p["cv2"] < params["cv2"] and p["n"]+1 <= maxFnEvals:
                params = p
        eta = np.concatenate(([params["c"]], np.array(params["a"]) + 1j*np.array(params["b"])))*params["mu1"]
        beta = np.concatenate(([1], 1 + 1j*np.arange(1,params["n"]+1)*params["omega"]))*params["mu1"]
    elif method=="euler":
        n_euler = math.floor((maxFnEvals-1)/2)
        eta = np.concatenate(([0.5], np.ones(n_euler), np.zeros(n_euler-1), [2**-n_euler]))
        logsum = np.cumsum(np.log(np.arange(1,n_euler+1)))
        for k in range(1,n_euler):
            eta[2*n_euler-k] = eta[2*n_euler-k + 1] + math.exp(logsum[n_euler-1] - n_euler*math.log(2.0) - logsum[k-1] - logsum[n_euler-k-1])
        k = np.arange(2*n_euler+1)
        beta = n_euler*math.log(10.0)/3.0 + 1j*math.pi*k
        eta  = (10**((n_euler)/3.0))*(1-(k%2)*2) * eta
    elif method=="gaver":
        if maxFnEvals%2==1:
            maxFnEvals -= 1
        ndiv2 = int(maxFnEvals/2)
        eta = np.zeros(maxFnEvals);
        beta = np.zeros(maxFnEvals);
        logsum = np.concatenate(([0], np.cumsum(np.log(np.arange(1,maxFnEvals+1)))))
        for k in range(1,maxFnEvals+1):
            inside_sum = 0.0;
            for j in range(math.floor((k+1)/2), min(k,ndiv2)+1):
                inside_sum += math.exp((ndiv2+1)*math.log(j) - logsum[ndiv2-j] + logsum[2*j] - 2*logsum[j] - logsum[k-j] - logsum[2*j-k]);
            eta[k-1] = math.log(2.0)*(-1)**(k+ndiv2)*inside_sum;
            beta[k-1] = k * math.log(2.0);
            
    res = [];
    for x in T:
        res.append(eta.dot([fun(b/x) for b in beta]).real/x)
    return res

## Demonstration of the usage

Some test functions:

In [2]:
tFuns = {"exponential": {
            "ilt": lambda x: math.exp(-x),
            "lt": lambda s: 1/(1+s),
            "xvals": np.arange(0.1, 5, 0.01),
            "yrange": [-0.5, 1.5]},
        "sine": {
            "ilt": lambda x: math.sin(x),
            "lt": lambda s: 1/(1+s**2),
            "xvals": np.arange(0.1, 15, 0.05),
            "yrange": [-1.2, 1.2]},
        "heavyside": {
            "ilt": lambda x: int(x>1),
            "lt": lambda s: cmath.exp(-s)/s,
            "xvals": np.arange(0.1, 3, 0.01),
            "yrange": [-0.5, 1.5]},
        "expheavyside": {
            "ilt": lambda x: int(x>1)*math.exp(1-x),
            "lt": lambda s: cmath.exp(-s)/(1+s),
            "xvals": np.arange(0.1, 5, 0.01),
            "yrange": [-0.5, 1.2]},
        "squarewave": {
            "ilt": lambda x: int(math.floor(x))%2,
            "lt": lambda s: (1.0/s)*(1.0/(1.0+cmath.exp(s))),
            "xvals": np.arange(0.1, 10, 0.02),
            "yrange": [-1.0, 2.0]},
        "staircase": {
            "ilt": lambda x: int(math.floor(x)),
            "lt": lambda s: (1.0/s)*(1.0/(cmath.exp(s)-1.0)),
            "xvals": np.arange(0.1, 5, 0.01),
            "yrange": [-1.0, 5.0]}
        }

The interactive demo:

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

@interact(funEvals=widgets.IntSlider(min=3,max=500,step=1,value=25), testFunction=["staircase","exponential","sine","heavyside","expheavyside","squarewave"])
def demo(funEvals, testFunction):
    euler = []
    gaver = []
    try:
        exact = [tFuns[testFunction]["ilt"](x) for x in tFuns[testFunction]["xvals"]]
        cme = ilt(tFuns[testFunction]["lt"], tFuns[testFunction]["xvals"], funEvals, "cme")
        euler = ilt(tFuns[testFunction]["lt"], tFuns[testFunction]["xvals"], funEvals, "euler")
        gaver = ilt(tFuns[testFunction]["lt"], tFuns[testFunction]["xvals"], funEvals, "gaver")
    except:
        pass
    plt.figure(figsize=(8, 6))
    plt.plot(exact, label="Exact")
    plt.plot(euler, label="Euler")
    plt.plot(gaver, label="Gaver")
    plt.plot(cme, label="CME")
    plt.ylim(tFuns[testFunction]["yrange"])
    plt.legend()

interactive(children=(IntSlider(value=25, description='funEvals', max=500, min=3), Dropdown(description='testF…